In [1]:
import numpy as np
import csv
import pandas as pd
import keras
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Using TensorFlow backend.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1w3nf2dE-fXLqReeIe3El99GgzsGr5Ecw'}) 
downloaded.GetContentFile('reddit_train.csv')    

reddit_train = pd.read_csv('reddit_train.csv')

In [0]:
def dataCompress(rawData):
  return rawData[:30000]

In [0]:
def encodeType(reddit_train):
  typeList = list()
  
  # extract subreddits
  for i in range(0,len(reddit_train)):  
    subreddit = reddit_train.iloc[i][2]
    #subreddit = reddit_train[i][2] 
    if subreddit not in typeList:
      typeList.append(subreddit)    
    # if(len(typeList)>=20):
    #   break;
    
  # index subreddits  
  encoded = list()
  index = 0;
  for element in typeList:
    my_tuple = (index,element)
    index = index+1
    encoded.append(my_tuple)
    
  return encoded

In [0]:
def typeSubreddit(reddit_train,subreddit_indexed):
  
  typeVector = list()
  
  for i in range(0,len(reddit_train)):  
    subreddit = reddit_train.iloc[i][2]
    
    for k in range(0,len(subreddit_indexed)):
      if subreddit_indexed[k][1] == subreddit:
        typeVector.append(subreddit_indexed[k][0])
   
  return typeVector

In [0]:
#reddit_train = dataCompress(reddit_train)
redditWords = reddit_train.T.iloc[1].to_numpy()

subreddit_indexed = encodeType(reddit_train)
redditLabel = typeSubreddit(reddit_train,subreddit_indexed)

In [7]:
import nltk
nltk.download('popular')
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def lemmatize_all(sentence):
    wnl = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(sentence)):
        if tag.startswith("NN"):
            yield wnl.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            yield wnl.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            yield wnl.lemmatize(word, pos='a')
        elif tag.startswith('R'):
            yield wnl.lemmatize(word, pos='r')
            
        else:
            yield word


def msgProcessing(raw_msg):
    m_w=[]
    words2=[]
    raw_msg=str(raw_msg)
    raw_msg = str(raw_msg.lower())
    #url_stripper= re.sub(r'Email me.*[A-Z]',"",s)
    
    #raw_msg=re.sub(r'\w*[0-9]\w*','', url_stripper)
    raw_msg=re.sub(r'[^a-zA-Z]', ' ', raw_msg)
    
    words=raw_msg.lower().split()
    #Remove words with length lesser than 3 if not w in stops
    for i in words:
        if len(i)>=1:
            words2.append(i)
    stops=set(stopwords.words('english'))
    m_w=" ".join([w for w in words2])
    return(" ".join(lemmatize_all(m_w)))


def helperFunction(df):
    print ("Data Preprocessing!!!")
    
    num_msg=df.shape[0]
    clean_msg=[]
    for i in range(0,num_msg):
        clean_msg.append(msgProcessing(df[i]))
    X=clean_msg
    print ("Data Preprocessing Ends!!!")
    return X

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [8]:
print(redditWords[5])
redditWords = helperFunction(redditWords)
print(redditWords[5])

The joke is on YOU!

I've only seen it twice... :/
Data Preprocessing!!!
Data Preprocessing Ends!!!
the joke be on you i ve only see it twice


In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# dataset = pd.read_csv('sentiment_analysis/data_train.csv', sep='\t',names=['ID', 'type', 'review', 'label']).astype(str)
# cw = lambda x: list(jieba.cut(x))
# dataset['words'] = dataset['review'].apply(cw)
tokenizer=Tokenizer()  #创建一个Tokenizer对象
#fit_on_texts函数可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小
x_train, x_test, y_train, y_test = train_test_split(redditWords, redditLabel, test_size=0.1)
tokenizer.fit_on_texts(x_train)
vocab=tokenizer.word_index #得到每个词的编号
print(len(vocab))

# 将每个样本中的每个词转换为数字列表，使用每个词的编号进行编号
x_train_word_ids=tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
#序列模式
# 每条样本长度不唯一，将每条样本的长度设置一个固定值
x_train_padded_seqs=pad_sequences(x_train_word_ids,maxlen=150) #将超过固定值的部分截掉，不足的在最前面用0填充
x_test_padded_seqs=pad_sequences(x_test_word_ids, maxlen=150)

56327


In [0]:
from sklearn import metrics
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D, BatchNormalization
def CNN_model(x_train_padded_seqs, y_train, x_test_padded_seqs, y_test):
    model = Sequential()
    model.add(Embedding(len(vocab) + 1, 300, input_length=150)) #使用Embeeding层将每个词编码转换为词向量
    model.add(Conv1D(256, 5, padding='same'))
    model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Conv1D(128, 5, padding='same'))
    model.add(MaxPooling1D(3, 3, padding='same'))
    model.add(Conv1D(64, 3, padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(BatchNormalization())  # (批)规范化层
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(20, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    one_hot_labels = keras.utils.to_categorical(y_train)  # 将标签转换为one-hot编码
    one_hot_labels_test = keras.utils.to_categorical(y_test)
    model.fit(x_train_padded_seqs, one_hot_labels,epochs=18, batch_size=16, validation_data=(x_test_padded_seqs, one_hot_labels_test))
    # y_predict = model.predict_classes(x_test_padded_seqs)  # 预测的是类别，结果就是类别号
    # y_predict = list(map(int, y_predict))
    # print('准确率', metrics.accuracy_score(y_test, y_predict))
    # print(y_test)
    # print(y_predict)
    #print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))

In [0]:
from sklearn import metrics
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D, BatchNormalization, concatenate
def TextCNN_model_1(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
    main_input = Input(shape=(150,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(len(vocab) + 1, 300, trainable=False)
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=48)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=47)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=46)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=-1)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(20, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
    one_hot_labels = keras.utils.to_categorical(y_train)  # 将标签转换为one-hot编码
    one_hot_labels_test = keras.utils.to_categorical(y_test)
    model.fit(x_train_padded_seqs, one_hot_labels, batch_size=64, epochs=16, validation_data=(x_test_padded_seqs, one_hot_labels_test))
    #xxxxxxy_test_onehot = keras.utils.to_categorical(y_test, num_classes=3)  # 将标签转换为one-hot编码
    #result = model.predict(x_test_padded_seqs)  # 预测样本属于每个类别的概率
    #result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签
    #y_predict = list(map(int, result_labels))
    #print('准确率', metrics.accuracy_score(y_test, y_predict))

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D, concatenate

def MLP(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
  model = Sequential()
  # 全连接层
  model.add(Dense(512, activation='relu'))
  # DropOut层
  model.add(Dropout(0.5))
  # 全连接层+分类器
  model.add(Dense(20, activation='softmax'))

  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  one_hot_labels_train = keras.utils.to_categorical(y_train)
  one_hot_labels_test = keras.utils.to_categorical(y_test)
  model.fit(x_train_padded_seqs, one_hot_labels_train, batch_size=100, epochs=15, validation_data=(x_test_padded_seqs, one_hot_labels_test))

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D, concatenate

def LSTMmodel(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
  model = Sequential()
  model.add(Embedding(len(vocab)+1, 300))
  model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.1))
  model.add(Dense(20, activation='softmax'))

  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  one_hot_labels_train = keras.utils.to_categorical(y_train)
  one_hot_labels_test = keras.utils.to_categorical(y_test)
  model.fit(x_train_padded_seqs, one_hot_labels_train, batch_size=64, epochs=15, validation_data=(x_test_padded_seqs, one_hot_labels_test))

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras import regularizers
from keras.engine.input_layer import Input
from keras.layers import LSTM, Activation, Dense, Dropout, Flatten, Embedding, Bidirectional, GRU, concatenate

def BiGRU(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
  model = Sequential()
  model.add(Embedding(len(vocab)+1, 300))
  model.add(Bidirectional(GRU(256, kernel_initializer='orthogonal', recurrent_regularizer=regularizers.l2(0.01), dropout=0.2, recurrent_dropout=0.1, return_sequences=True)))
  model.add(Bidirectional(GRU(256, kernel_initializer='orthogonal', recurrent_regularizer=regularizers.l2(0.01), dropout=0.2, recurrent_dropout=0.1)))
  model.add(Dense(20, activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  one_hot_labels_train = keras.utils.to_categorical(y_train)
  one_hot_labels_test = keras.utils.to_categorical(y_test)
  model.fit(x_train_padded_seqs, one_hot_labels_train, batch_size=32, epochs=15, validation_data=(x_test_padded_seqs, one_hot_labels_test))

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import GRU, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D, concatenate, Bidirectional

def CNNRNN(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
  main_input = Input(shape=(150,), dtype='float64')
  embed = Embedding(len(vocab)+1, 300)(main_input)
  cnn = Conv1D(256, 3, padding='same', strides = 1, activation='relu')(embed)
  cnn = MaxPooling1D(pool_size=4)(cnn)
  cnn = Flatten()(cnn)
  cnn = Dense(256)(cnn)
  rnn = Bidirectional(GRU(256, dropout=0.2, recurrent_dropout=0.1))(embed)
  rnn = Dense(256)(rnn)
  con = concatenate([cnn,rnn], axis=-1)
  main_output = Dense(20, activation='softmax')(con)
  model = Model(inputs = main_input, outputs = main_output)
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  one_hot_labels_train = keras.utils.to_categorical(y_train)
  one_hot_labels_test = keras.utils.to_categorical(y_test)
  model.fit(x_train_padded_seqs, one_hot_labels_train, batch_size=32, epochs=10, validation_data=(x_test_padded_seqs, one_hot_labels_test))

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import regularizers
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import GRU, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D, concatenate, BatchNormalization

def CNNLSTM(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
  model = Sequential()
  model.add(Embedding(len(vocab)+1, 300, input_length=150))
  model.add(Conv1D(32,kernel_size=3,padding='same',activation='relu'))
  model.add(MaxPooling1D(pool_size=3))
  model.add(Dropout(0.3))
  model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
  model.add(MaxPooling1D(pool_size=3))
  model.add(Dropout(0.35))
  model.add(Conv1D(128,kernel_size=3,padding='same',activation='relu'))
  model.add(MaxPooling1D(pool_size=3))
  model.add(Dropout(0.4))
  model.add(GRU(256, kernel_initializer='orthogonal', recurrent_regularizer=regularizers.l2(0.01), dropout=0.2, recurrent_dropout=0.1, return_sequences = True))
  model.add(GRU(256, kernel_initializer='orthogonal', recurrent_regularizer=regularizers.l2(0.01), dropout=0.2, recurrent_dropout=0.1))
  model.add(Dropout(0.25))
  #model.summary()
  #model.add(Flatten())
  model.add(Dense(128,activation='relu'))
  model.add(Dropout(0.45))
  model.add(Dense(20, activation='softmax'))
  
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

  one_hot_labels_train = keras.utils.to_categorical(y_train)
  one_hot_labels_test = keras.utils.to_categorical(y_test)
  model.fit(x_train_padded_seqs, one_hot_labels_train, batch_size=128, epochs=20, validation_data=(x_test_padded_seqs, one_hot_labels_test))

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.models import Model
from keras.engine.input_layer import Input
from keras.layers import GRU, Activation, Dense, Dropout, Flatten, Embedding, Conv1D, MaxPooling1D
from keras.layers import concatenate, Bidirectional, BatchNormalization
from keras.utils import plot_model
#from google.colab import files

def Trinity(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test):
  main_input = Input(shape=(150,), dtype='float64')
  embed = Embedding(len(vocab)+1, 300)(main_input)

  cnnlstm = Conv1D(32,kernel_size=3,padding='same',activation='relu')(embed)
  cnnlstm = MaxPooling1D(pool_size=3)(cnnlstm)
  cnnlstm = Dropout(0.3)(cnnlstm)
  cnnlstm = Conv1D(64,kernel_size=3,padding='same',activation='relu')(cnnlstm)
  cnnlstm = MaxPooling1D(pool_size=3)(cnnlstm)
  cnnlstm = Dropout(0.35)(cnnlstm)
  cnnlstm = Conv1D(128,kernel_size=3,padding='same',activation='relu')(cnnlstm)
  cnnlstm = MaxPooling1D(pool_size=3)(cnnlstm)
  cnnlstm = Dropout(0.4)(cnnlstm)
  cnnlstm = GRU(256, dropout=0.2, recurrent_dropout=0.1, return_sequences = True)(cnnlstm)
  cnnlstm = GRU(256, dropout=0.2, recurrent_dropout=0.1, go_backwards=True)(cnnlstm)
  cnnlstm = Dropout(0.25)(cnnlstm)
  cnnlstm = Dense(128)(cnnlstm)
  
  cnn = Conv1D(256, 5, padding='same')(embed)
  cnn = MaxPooling1D(3, 3, padding='same')(cnn)
  cnn = Conv1D(128, 5, padding='same')(cnn)
  cnn = MaxPooling1D(3, 3, padding='same')(cnn)
  cnn = Conv1D(64, 3, padding='same')(cnn)
  cnn = Flatten()(cnn)
  cnn = Dropout(0.1)(cnn)
  cnn = BatchNormalization()(cnn)
  cnn = Dense(128)(cnn)


  bigru = Bidirectional(GRU(256, kernel_initializer='orthogonal', recurrent_regularizer=regularizers.l2(0.01), dropout=0.2, recurrent_dropout=0.1, return_sequences=True))(embed)
  bigru = Bidirectional(GRU(256, kernel_initializer='orthogonal', recurrent_regularizer=regularizers.l2(0.01), dropout=0.2, recurrent_dropout=0.1))(bigru)
  bigru = Dense(128)(bigru)

  
  con = concatenate([cnnlstm,cnn,bigru], axis=-1)
  main_output = Dense(20, activation='softmax')(con)
  model = Model(inputs = main_input, outputs = main_output)
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()
  # plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
  # uploaded = drive.CreateFile({'title': 'abc.png'})
  # uploaded.SetContentFile('model.png')
  # uploaded.Upload() 

  one_hot_labels_train = keras.utils.to_categorical(y_train)
  one_hot_labels_test = keras.utils.to_categorical(y_test)
  model.fit(x_train_padded_seqs, one_hot_labels_train, batch_size=64, epochs=15, validation_data=(x_test_padded_seqs, one_hot_labels_test))

In [26]:
Trinity(x_train_padded_seqs, y_train, x_test_padded_seqs, y_test)

Train on 63000 samples, validate on 7000 samples
Epoch 1/1
63000/63000 [==============================] - 801s 13ms/step - loss: 2.5789 - acc: 0.3325 - val_loss: 1.9196 - val_acc: 0.4574
